In this journal, we aim to train a functioning audio sentiment model. This model would be utilised in an analyser, which can be represented as:

Audio Object -------> Sentiment Analyser ------> Sentiment Object

The file names for each audio file describes various details about the short audio clip that comes with it, including the emotion. We will use emotion attached to train the model.

In [17]:
import os
from fetch_emotion import get_emotion
import pandas as pd
import numpy as np
import librosa
import resampy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, AveragePooling1D, BatchNormalization, Input, Flatten, Dropout, Activation
from keras.utils import to_categorical, np_utils

2023-10-01 23:50:02.619331: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 23:50:02.721967: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-01 23:50:02.722050: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-01 23:50:02.722228: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-01 23:50:02.759601: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-01 23:50:02.760809: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

ImportError: cannot import name 'np_utils' from 'keras.utils' (/home/benben1752002/anaconda3/lib/python3.11/site-packages/keras/utils/__init__.py)

In [4]:
path = '../audiofiles'

Here, we want to have all the files available for training.

In [7]:
file_names = []
for path, _, files in os.walk(path):
    for name in files:
        if name[-3:] == 'wav':
            file_names.append(os.path.join(path, name))

Audio features have to be extracted from each audio sample into a data frame.

In [10]:
df = pd.DataFrame(columns=['feature', 'emotion'])

for i, name in enumerate(file_names):
    X, sample_rate = librosa.load(name, res_type='kaiser_fast', duration=3, sr=22050*2, offset=0.5)
    sample_rate = np.array(sample_rate)
    # Get features of audio sample
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)
    df.loc[i] = [mfccs, get_emotion(name)]

Preprocess into suitable format.

In [15]:
feature_df = pd.DataFrame(df['feature'].values.tolist())
labelled_df = pd.concat([feature_df, df['emotion']], axis=1)
labelled_df = labelled_df.dropna()
shuffled_df = labelled_df.sample(frac=1).reset_index(drop=True)
shuffled_df = shuffled_df.loc[~shuffled_df['emotion'].isin(['neutral', 'disgusted', 'surprised'])]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(shuffled_df.drop('emotion', axis =1), shuffled_df['emotion'], test_size=0.2, random_state=0, shuffle = True)

Now, we want to encode the labels for the training and test data locally.

In [21]:
encoder = LabelEncoder()
y_train = np_utils.to_categorical(encoder.fit_transform(y_train))
y_test = np_utils.to_categorical(encoder.fit_transform(y_test))
np.save('label_classes.npy', encoder.classes_)

NameError: name 'np_utils' is not defined